## Chapter 20: Simulating Poker Hands

This chapter uses the ideas from the previous two chapters to study the probability that certain poker hands arise.  

In this chapter, we are only concerned with 5 cards with no jokers and just determining if a hand satisfies one of the following:

* _Royal Flush_: the ranks of the cards are 10, J, Q, K, A and all cards have the same suit.
* _Straight Flush_: the ranks are sequential and all cards have the same suit. We will allow Ace to be both high (as in a Royal Flush) and low, like, A, 2, 3, 4, 5.
* _Flush_: All cards have the same suit. We will exclude straight flushes, but ace can be high or low.
* _Straight_: The ranks of the 5 cards are sequential. Again, we will exclude straight flushes.
* _Four of a kind_: four of the cards have the same rank
* _Full House_: two cards have the same rank, the other three cards have the same rank. The suit doesn’t matter.
* _Three of a kind_: three of the cards have the same rank. The other two cards do not have the same rank. The suit of the cards doesn’t matter and also, make sure that the other two cards are not a pair or that would be a full house.
* _Two pairs_: Two cards have the same rank. Two of the remaining cards have the same rank, but different than the first two pair. The 5th card does not make it a full house.
* One pair: two cards have the same rank. The remaining cards do not make it a different type of hand (full house, three of a kind, etc.)
* _No pair_ or _nothing_: the cards don’t form any other hand. This is also called _High Card_, in that if comparing hands, the highest card in the hand is important.

The following will load in a package that I made.  We will see how to create a package shortly.  Note: because the package is local (instead of being fetched from the internet), the name of the package in the `using` statement is proceded by a . 

In [1]:
include("../julia-files/PlayingCards.jl")
using .PlayingCards

Here's a list of those hands above using a different Hand constructor.

In [2]:
royal_flush_1 = Hand("T♠,J♠,Q♠,K♠,A♠")

[T♠,J♠,Q♠,K♠,A♠]

The suits can be entered in by typing `\heartsuit` or `\spadesuit`, etc.  then hit TAB. 

In [3]:
straight_flush_1 = Hand("2♡,3♡,4♡,5♡,6♡")

[2♡,3♡,4♡,5♡,6♡]

In [4]:
flush_1 = Hand("2♣,7♣,T♣,3♣,A♣")

[2♣,7♣,T♣,3♣,A♣]

In [5]:
straight_1 = Hand("4♣,5♠,6♠,7♢,8♣")

[4♣,5♠,6♠,7♢,8♣]

In [6]:
four_of_a_kind_1 = Hand("2♠,5♠,5♢,5♣,5♡")

[2♠,5♠,5♢,5♣,5♡]

In [7]:
full_house_1 = Hand("4♣,4♢,4♠,7♣,7♢")
full_house_2 = Hand("4♣,4♢,7♠,7♣,7♢")

[4♣,4♢,7♠,7♣,7♢]

In [8]:
three_of_a_kind_1 = Hand("4♠,4♢,4♣,A♠,7♡")

[4♠,4♢,4♣,A♠,7♡]

In [9]:
two_pair_1 = Hand("4♣,4♢,5♠,7♣,7♢")

[4♣,4♢,5♠,7♣,7♢]

In [10]:
one_pair_1 = Hand("2♠,2♢,5♣,8♢,Q♣")

[2♠,2♢,5♣,8♢,Q♣]

In [11]:
nothing_hand = Hand("T♣,4♢,Q♠,3♣,6♢")

[T♣,4♢,Q♠,3♣,6♢]

If we are to study the probability that poker hands arise, then we will want functions that test a hand.  For example, let's look a one that tests for a full house.  This hand is probably the easiest to test because it takes all 5 cards and there are special cases (like straights, straight flushes, royal flushes)

In [12]:
isFullHouse(full_house_1)

true

In [13]:
isFullHouse(full_house_2)

true

In [14]:
isFullHouse(Hand("2♠,5♢,2♣,5♠,5♡"))

true

In [16]:
isFullHouse(Hand("5♠,5♢,5♣,4♠,5♡"))

false

Let's now test some hands

In [33]:
isFullHouse(full_house_1),isFullHouse(full_house_2),isFullHouse(three_of_a_kind_1)

(true, true, false)

### 20.2: Simulating Poker Hands

We are probably interested in determining how often a given poker hand comes up.  There are counting techniques to do this, but we will do a simulation instead.  Here's the idea:

1. Take a deck and shuffle it. 
2. Use the top 5 cards as a hand
3. test if the hand is a full house, straight, etc. 
4. if it is, tally that. 
5. Repeat this a large number of times.

The following will do this.

In [18]:
using Random
function runTrials(f::Function, trials::Integer)
  local deck=collect(1:52) # creates the array [1,2,3,...,52]
  local num_hands=0
  for i=1:trials
    shuffle!(deck)
    h = Hand(map(Card,deck[1:5])) # creates a hand of the first five cards of the shuffled deck
    if f(h)
      num_hands+=1
    end
  end
  num_hands/trials
end

runTrials (generic function with 1 method)

We can now run this on the `isFullHouse` function

In [19]:
runTrials(isFullHouse,10_000_000)

0.0014521

Check out the [wikipedia page on Poker Probabilities](https://en.wikipedia.org/wiki/Poker_probability). Using counting techniques (that is determining the total number of full house hands divided by the total number of 5 card hands), it is 0.1441%. 

#### 20.3: Probabilities of Other Hands

I mentioned above that some of the hands are difficult to determine because you need to exclude some other hands.  For example, a flush has all the same suit (but is not a straight flush or royal flush). To help with this, we're going to produce two functions that test all the same suit, and if there is a run. 

In [20]:
function isOneSuit(h::Hand)
  local s = map(c->c.suit,h.cards)
  s[1]==s[2]==s[3]==s[4]==s[5]
end

isOneSuit (generic function with 1 method)

In [21]:
isOneSuit(flush_1)

true

In [22]:
isOneSuit(Hand("2♣,6♣,A♣,J♣,3♠"))

false

In [23]:
function isRun(h::Hand)
  local r = sort(map(c->c.rank,h.cards))
  r[2]==r[1]+1 && r[3]==r[2]+1 && r[4]==r[3]+1 && r[5]==r[4]+1 ||
  r[1]==1 && r[2]==10 && r[3]==11 && r[4]==12 && r[5]==13 ## ace high run
end

isRun (generic function with 1 method)

In [24]:
isRun(Hand("A♣,2♣,3♢,4♡,5♡"))

true

In [25]:
function isRoyalFlush(h::Hand)
  local r = sort(map(c->c.rank,h.cards))
  r[1]==1 && r[2]==10 && r[3]==11 && r[4]==12 && r[5]==13 && isOneSuit(h)
end

isRoyalFlush (generic function with 1 method)

In [26]:
royal_flush_1

[T♠,J♠,Q♠,K♠,A♠]

In [28]:
isRoyalFlush(royal_flush_1)

true

In [29]:
isRoyalFlush(straight_flush_1)

false

In [31]:
@time runTrials(isRoyalFlush,10_000_000)

  4.515428 seconds (80.01 M allocations: 4.024 GiB, 5.97% gc time)


9.0e-7

The actual probability is 0.00000154 (or 0.000154%)

#### Two-Pair Function

In [71]:
function isTwoPair(h::Hand)

end

isTwoPair (generic function with 1 method)

In [72]:
two_pair_1

[4♣,4♢,5♠,7♣,7♢]

In [79]:
isTwoPair(two_pair_1)

true

In [78]:
isTwoPair(four_of_a_kind_1)

true

In [77]:
isTwoPair(full_house_1)

false

Note that if you have a four of a kind this should not work:

In [ ]:
isTwoPair(four_of_a_kind_1)